<a href="https://colab.research.google.com/github/siddhantkd/HGR-Net/blob/master/HGR_Net_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DATALOADER

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip '/content/drive/MyDrive/OUHANDS/OUHANDS_Siddhant.zip'

Streaming output truncated to the last 5000 lines.
  inflating: OUHANDS_Siddhant/testing/segmentation/A-aha-0005.png  
  inflating: OUHANDS_Siddhant/testing/segmentation/A-aha-0006.png  
  inflating: OUHANDS_Siddhant/testing/segmentation/A-aha-0007.png  
  inflating: OUHANDS_Siddhant/testing/segmentation/A-aha-0008.png  
  inflating: OUHANDS_Siddhant/testing/segmentation/A-aha-0009.png  
  inflating: OUHANDS_Siddhant/testing/segmentation/A-aha-0010.png  
  inflating: OUHANDS_Siddhant/testing/segmentation/A-apa-0001.png  
  inflating: OUHANDS_Siddhant/testing/segmentation/A-apa-0002.png  
  inflating: OUHANDS_Siddhant/testing/segmentation/A-apa-0003.png  
  inflating: OUHANDS_Siddhant/testing/segmentation/A-apa-0004.png  
  inflating: OUHANDS_Siddhant/testing/segmentation/A-apa-0005.png  
  inflating: OUHANDS_Siddhant/testing/segmentation/A-apa-0006.png  
  inflating: OUHANDS_Siddhant/testing/segmentation/A-apa-0007.png  
  inflating: OUHANDS_Siddhant/testing/segmentation/A-apa-0008.png

In [3]:
import cv2
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import keras
from tqdm import tqdm

In [4]:
IMG_WIDTH = 320
IMG_HEIGHT = 320
IMG_CHANNELS = 3

In [5]:
path = "/content/OUHANDS_Siddhant/training/colour"
maskpath = "/content/OUHANDS_Siddhant/training/segmentation"

X_train = np.zeros((len(os.listdir(path)), IMG_HEIGHT, IMG_WIDTH,IMG_CHANNELS),dtype=np.uint8)
Y_train = np.zeros((len(os.listdir(maskpath)), IMG_HEIGHT, IMG_WIDTH,1))
print('creating input data')
n=0
for name in tqdm(os.listdir(path)):
    newpath = os.path.join(path,name)
    img=cv2.imread(newpath)
    img=cv2.resize(img,(IMG_WIDTH,IMG_HEIGHT))
    X_train[n]=img
    n+=1

m=0
for name in tqdm(os.listdir(maskpath)):
    newmaskpath = os.path.join(maskpath,name)
    mpg=cv2.imread(newmaskpath,0)
    mpg=cv2.resize(mpg,(IMG_WIDTH,IMG_HEIGHT))
    mpg=np.expand_dims(mpg,axis=-1)
    Y_train[m]=mpg/255
    m+=1

creating input data


100%|██████████| 2000/2000 [00:03<00:00, 581.84it/s]


In [6]:
X_train.shape

(2000, 320, 320, 3)

SEGMENTATION 

In [7]:
#Import necessary libraries

import keras

import tensorflow as tf
from keras.datasets import mnist

from keras.models import Model,Input

from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,AveragePooling2D,Dropout,BatchNormalization,Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
from keras import backend as K

In [8]:
# Defining the Segmentation Model



class SegModel(object):
        def __init__(self, input_size):
            self.input_size=input_size
            self._build_model()
            

        def relu(self,x):
            return Activation('relu')(x)

        def ResidualNet(self,filters,s):
            def Res_unit(x):
              BottleN = int(filters / 4)
              b_filter = BottleN

              x = BatchNormalization(axis=-1)(x)
              x = Activation("relu")(x)
              ident_map = x

              x = Conv2D(filters=b_filter, kernel_size=(1, 1), strides=(s, s))(x)
              
              x = BatchNormalization(axis=-1)(x)
              x = Activation("relu")(x)

              x = Conv2D(filters=b_filter, kernel_size=(3, 3),  padding="same")(x)

              x = BatchNormalization(axis=-1)(x)
              x = Activation("relu")(x)

              x = Conv2D(filters=filters, kernel_size=(1,1))(x)

              ident_map = Conv2D(filters,(1,1),strides=(s,s))(ident_map)
              
              out = keras.layers.add([ident_map,x])

              return out
            return Res_unit

        def Res_Group(self,nfilter,layers,_stride):
            def Res_unit(x):
                for i in range(layers):
                    if i==0:
                        x = self.ResidualNet(nfilter,_stride)(x)
                    else:
                        x = self.ResidualNet(nfilter,1)(x)
                                   
                              
                return x
            return Res_unit

        #-------------------Pyramid Dilated Convolution--------------------------------
        def ASPP(self,input_layer):
            l = BatchNormalization(axis=-1)(input_layer)
            l = self.relu(l)
            conv1 = l
            dconv_filters=32
            a1 = Conv2D(dconv_filters, 1, activation = 'relu', padding = 'same', dilation_rate = 1)(conv1)
            a2 = Conv2D(dconv_filters, 3, activation = 'relu', padding = 'same', dilation_rate = 3)(conv1)
            a3 = Conv2D(dconv_filters, 3, activation = 'relu', padding = 'same', dilation_rate = 6)(conv1)
            a4 = Conv2D(dconv_filters, 3, activation = 'relu', padding = 'same', dilation_rate = 12)(conv1)
            a5 = Conv2D(dconv_filters, 3, activation = 'relu', padding = 'same', dilation_rate = 18)(conv1)

            concat = keras.layers.concatenate([a1,a2,a3,a4,a5], axis = 3)  
            
            return concat


        def _build_model(self):
        #--------------------encoder---------    
            inp = Input(shape=(self.input_size))
            i = inp
            i = Conv2D(16,7,padding='same')(i)
        #----------------------------------------
            i = self.Res_Group(32,3,1)(i) 
        #----------------------------------------
            i = self.Res_Group(64,3,2)(i) 
        #---------------------------------------
            i = self.Res_Group(128,3,2)(i) 
            out_pdc2 = self.ASPP(i)
        #---------------------------------------

        #-----------------------decoder----------------
            i_dec = Dropout(0.15)(out_pdc2)

            conv_f = Conv2D(1,(1, 1), activation='sigmoid', padding='same')(i_dec)

            i_dec=keras.layers.UpSampling2D(size = (4,4), interpolation='bilinear')(conv_f)



            model = Model(inputs=inp, outputs=i_dec )

            self.model=model

Training the Model

In [9]:
input_size = (320,320,3)
SegM=SegModel(input_size)
model=SegM.model
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 320, 320, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 320, 320, 16) 2368        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 320, 320, 16) 64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 320, 320, 16) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [10]:
_adam = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
)

In [11]:
model.compile(loss='binary_crossentropy',optimizer = _adam,metrics=['accuracy'])

In [15]:
#Model Checkpoint

checkpoint_path = "/content/drive/MyDrive/chekp"

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                  monitor='val_accuracy',save_best_only=True,
                                                 verbose=1)

In [16]:
#TRAIN

model.fit(X_train, Y_train,validation_split=0.1, batch_size=8, epochs=10, verbose=1,callbacks=[model_checkpoint])

Epoch 1/10
225/225 [==============================] - 130s 577ms/step - loss: 0.0325 - accuracy: 0.9846 - val_loss: 0.0489 - val_accuracy: 0.9790

Epoch 00001: val_accuracy improved from -inf to 0.97901, saving model to /content/drive/MyDrive/chekp
Epoch 2/10
225/225 [==============================] - 129s 572ms/step - loss: 0.0312 - accuracy: 0.9851 - val_loss: 0.1158 - val_accuracy: 0.9499

Epoch 00002: val_accuracy did not improve from 0.97901
Epoch 3/10
225/225 [==============================] - 130s 576ms/step - loss: 0.0309 - accuracy: 0.9853 - val_loss: 0.0356 - val_accuracy: 0.9835

Epoch 00003: val_accuracy improved from 0.97901 to 0.98352, saving model to /content/drive/MyDrive/chekp
Epoch 4/10
225/225 [==============================] - 130s 576ms/step - loss: 0.0339 - accuracy: 0.9840 - val_loss: 0.0442 - val_accuracy: 0.9800

Epoch 00004: val_accuracy did not improve from 0.98352
Epoch 5/10
225/225 [==============================] - 129s 572ms/step - loss: 0.0306 - accuracy

In [17]:
# Loads the weights
model.load_weights(checkpoint_path)

In [18]:
model.fit(X_train, Y_train,validation_split=0.1, batch_size=8, epochs=2, verbose=1,callbacks=[model_checkpoint])

Epoch 1/2
225/225 [==============================] - 130s 578ms/step - loss: 0.0283 - accuracy: 0.9862 - val_loss: 0.0831 - val_accuracy: 0.9658

Epoch 00001: val_accuracy did not improve from 0.98514
Epoch 2/2
225/225 [==============================] - 130s 576ms/step - loss: 0.0283 - accuracy: 0.9863 - val_loss: 0.0408 - val_accuracy: 0.9812

Epoch 00002: val_accuracy did not improve from 0.98514


EVALUATION

In [22]:
path = "/content/OUHANDS_Siddhant/testing/colour"
maskpath = "/content/OUHANDS_Siddhant/testing/segmentation"

X_test = np.zeros((len(os.listdir(path)), IMG_HEIGHT, IMG_WIDTH,IMG_CHANNELS),dtype=np.uint8)
Y_test = np.zeros((len(os.listdir(maskpath)), IMG_HEIGHT, IMG_WIDTH,1))
print('creating test data')
n=0
for name in tqdm(os.listdir(path)):
    newpath = os.path.join(path,name)
    img=cv2.imread(newpath)
    img=cv2.resize(img,(IMG_WIDTH,IMG_HEIGHT))
    X_test[n]=img
    n+=1

m=0
for name in tqdm(os.listdir(maskpath)):
    newmaskpath = os.path.join(maskpath,name)
    mpg=cv2.imread(newmaskpath,0)
    mpg=cv2.resize(mpg,(IMG_WIDTH,IMG_HEIGHT))
    mpg=np.expand_dims(mpg,axis=-1)
    Y_test[m]=mpg/255
    m+=1

creating test data


100%|██████████| 1000/1000 [00:02<00:00, 479.38it/s]


In [27]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.metrics import f1_score


In [28]:
def fscore(tp,Images,Masks):
        

    total=0
    i=0
    fs=0
    for i in range(len(Images)):
        total += 1


        tp[i][tp[i]>0.5]=1
        tp[i][tp[i]<0.5]=0

        pred = img_to_array(tp[i]).astype(int)
        label = img_to_array(np.squeeze(Masks[i], axis=2)).astype(int)
        flat_pred = np.ravel(pred)
        flat_label = np.ravel(label)

        fs += f1_score(flat_label,flat_pred, average='micro')

    fs=fs/total

    return fs  

In [29]:
 #    ######evaluate#####
model.load_weights(checkpoint_path)
y_pred=model.predict(X_test)
f=fscore(y_pred,X_test,Y_test)
print(f)

0.9639694335937498
